In [490]:
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures


In [491]:
 #Activity 3. Fitlinear regression using the closed-form solution. 

boston = load_boston()

X= boston.data

Y = boston.target

def determine_theta (x,y):
    #Append a column of ones in X to add the bias term
    x = np.c_[np.ones((x.shape[0], 1)), x]
    n_train = x.shape[0] #Number of tarining exmaples
    #reshaping y to (m,1)
    y=y.reshape(n_train,1)
    #Normal equation
    theta = np.dot (np.linalg.inv(np.dot(x.T,x)), np.dot(x.T, y))
    return theta

def predict (x, theta) :  
    #This could be worng as well
    x = np.c_[np.ones((x.shape[0], 1)), x]
    #preds is y_hat
    preds = np.dot(x, theta)
    return preds

# Mean Squared Error (MSE)
def mean_squared_error(y_pred, y_true):
    # Number of samples
    n = len(y_true)
    # Calculate the squared differences
    squared_diffs = (y_pred - y_true) ** 2
    # Compute the average of the squared differences (MSE)
    mse = np.mean((squared_diffs))
    return mse


/home/nyuad/anaconda3/envs/virtualenv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

In [492]:
# Activity 3. Use k fold cross validation to estimate the performance of the model
# Initialize lists to store the training and test scores

def k_fold_func (x, y) :
    kf= KFold(n_splits = 10)
    train_errors = []
    test_errors = []
    # Loop over each fold
    for train_index, test_index in kf.split(X):
        # Split the data into training and test sets
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        theta = determine_theta (X_train, y_train)
        #predictions
        y_train_pred = predict (X_train, theta)
        y_test_pred = predict (X_test, theta)
        # Compute the Mean Squared Error (MSE) for both training and test sets -> Need to actually write formula
        train_mse = mean_squared_error(y_train_pred, y_train) #mean_squared_error(y_train, y_train_pred)
        test_mse = mean_squared_error(y_test_pred, y_test)
        # Append the errors to the lists
        train_errors.append(train_mse)
        test_errors.append(test_mse)
    # Calculate the average error across all folds
    avg_train_error = np.mean(train_errors)
    avg_test_error = np.mean(test_errors)
    print(f"Average training MSE: {avg_train_error}")
    print(f"Average test MSE: {avg_test_error}")

#Need to correct this
#k_fold_func(X, Y)

#Ask about the difference between loss and error like which arguments should we input into the MSE function

In [493]:
#Activity 3 Print the average recorded socres for both the test and the training set
k_fold_func(X, Y)

Average training MSE: 146.62656490039512
Average test MSE: 84.68254533486471


In [494]:
#Activity 4 Fit a ridge regression model using the closed solution

# Use k-fold cross-validation to findthe best λUse the Numpy function:
#np.logspace(1, 7, num=13) to get the differentvalues for λ

def Ridge_Regression_closed(x, y, lambd):
    alpha = lambd #lambda
    x_b = np.c_[np.ones((x.shape[0], 1)), x]
    I = np.identity (x_b.shape[1])
    I[0][0]=0
    #Implementing the penalty eq
    penalty = alpha*I    
    #Equation to solve for b
    b= np.linalg.inv((x_b.T @ x_b) + penalty)@(x_b.T@y)
    return b

def predict_ridge (x, theta) :  
    #X = np.c_[np.ones((X.shape[0], 1)), X]
    #x = np.c_[np.ones((x.shape[0], 1)), x]
    #preds is y_hat
    #preds = np.dot(x, theta)
    X_b = np.c_[np.ones((x.shape[0], 1)), x]  # Add bias term
    return np.dot(X_b, theta)
    #return preds

#cross validation to determine the best lambda
def ridge_cross_validation_best (x,y,op_lamb):
    kf = KFold(n_splits=10)
    avg_losses = [] # ask what should go into this list
    avg_train_error = []
    avg_test_error = []
    train_errors = []
    test_errors = []
    #lambda_pairs = []
    best_mse = float('inf')
    for iterator in op_lamb:
        for train_index, test_index in kf.split(x):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]
            theta = Ridge_Regression_closed(X_train, y_train, iterator)
            #predictions
            y_train_pred = predict_ridge (X_train, theta)
            y_test_pred = predict_ridge (X_test, theta)
            # Compute the Mean Squared Error (MSE) for both training and test sets -> Need to actually write formula
            train_mse = mean_squared_error(y_train_pred, y_train)
            test_mse = mean_squared_error(y_test_pred, y_test) #actual value loss
            # Append the errors to the lists
            train_errors.append(train_mse)
            test_errors.append(test_mse)
        # Calculate the average error across all folds
        avg_train_error.append(np.mean(train_errors))
        avg_test_error.append(np.mean(test_errors)) # for each one saved it corresponds to one of the op_lamb
        #lambda_pairs.append((iterator, avg_test_error))
        #if avg_test_error < best_mse:
          #  best_mse = avg_test_error
           # best_lambda = lmbda
    #best_lambda_index = np.argmin(avg_test_error)
    #best_lambda = op_lamb[best_lambda_index]
      # Track the best lambda by finding the lowest test error
        if np.mean(test_errors) < best_mse:
            best_mse = np.mean(test_errors)
            best_lambda = iterator
    
    return best_lambda

#performance of ridge regression with a determined lamb
def ridge_cross_validation (x, y, lamb) :
    kf= KFold(n_splits = 10)
    train_errors = []
    test_errors = []
    # Loop over each fold
    for train_index, test_index in kf.split(X):
        # Split the data into training and test sets
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        theta = Ridge_Regression_closed(X_train, y_train, lamb)
        #predictions
        y_train_pred = predict_ridge (X_train, theta)
        y_test_pred = predict_ridge (X_test, theta)
        # Compute the Mean Squared Error (MSE) for both training and test sets -> Need to actually write formula
        train_mse = mean_squared_error(y_train_pred, y_train)
        test_mse = mean_squared_error(y_test_pred, y_test)
        # Append the errors to the lists
        train_errors.append(train_mse)
        test_errors.append(test_mse)

    # Calculate the average error across all folds
    avg_train_error = np.mean(train_errors)
    avg_test_error = np.mean(test_errors)
    print(f"Average training MSE: {avg_train_error}")
    print(f"Average test MSE: {avg_test_error}")


In [495]:
#Activity 5 For the best λ you found, use k-fold cross-validation to estimate the performance of thismodel with this λ
# code to try functions
op_lamb = np.logspace(1, 7, num=13)
best_lamb = ridge_cross_validation_best (X,Y,op_lamb)
print(f"Best lambda value was: {best_lamb}")
ridge_cross_validation(X,Y,best_lamb)

Best lambda value was: 316.22776601683796
Average training MSE: 25.374775995777483
Average test MSE: 29.799581632087218


In [496]:
#Activity 5. Print the average of your recorded scores for both the test set andtraining set.
#Done above


In [497]:
#Activity 6. Repeat the previous exercise, but this time, by creating a polynomial transformation of degree 2 on the features of the
#dataset.a.Using PolynomialFeatures(degree=2) in sklearn library7  

poly_features = PolynomialFeatures (degree=2, include_bias =False)
x_poly = poly_features.fit_transform (X)

def Ridge_Regression_closed_poly(x, y, lambd):
    alpha = lambd #lambda
    x_columns1 = np.c_[np.ones((x.shape[0], 1)), x]
    I = np.identity (x_columns1.shape[1])
    I[0][0]=0
    #Implementing the penalty eq
    penalty = alpha*I    
    #Equation to solve for b
    b= np.linalg.inv((x_columns1.T @ x_columns1) + penalty)@(x_columns1.T@y)
    return b

def predict_ridge_poly (x, theta) :  
    X_predictor = np.c_[np.ones((x.shape[0], 1)), x]
    #preds is y_hat
    preds = np.dot(X_predictor, theta)
    return preds

def ridge_cross_validation_best_poly (x,y,op_lamb):
    kf = KFold(n_splits=10)
    avg_losses = [] # ask what should go into this list
    avg_train_error = []
    avg_test_error = []
    train_errors = []
    test_errors = []
    #poly_param = []
    for iterator in op_lamb:
        for train_index, test_index in kf.split(x):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]
            theta = Ridge_Regression_closed_poly(X_train, y_train, iterator)
            #poly_param.append(theta)
            #predictions
            y_train_pred = predict_ridge_poly (X_train, theta)
            y_test_pred = predict_ridge_poly (X_test, theta)
            # Compute the Mean Squared Error (MSE) for both training and test sets -> Need to actually write formula
            train_mse = mean_squared_error(y_train_pred, y_train)
            test_mse = mean_squared_error(y_test_pred, y_test) #actual value loss
            # Append the errors to the lists
            train_errors.append(train_mse)
            test_errors.append(test_mse)
        # Calculate the average error across all folds
        avg_train_error.append(np.mean(train_errors))
        avg_test_error.append(np.mean(test_errors)) # for each one saved it corresponds to one of the op_lamb
    best_lambda_index = np.argmin(avg_test_error)
    best_lambda = op_lamb[best_lambda_index]
    return best_lambda

#performance of ridge regression with a determined lamb
def ridge_cross_validation_poly (x, y, lamb) :
    kf= KFold(n_splits = 10)
    train_errors = []
    test_errors = []
    poly_param =[]
    # Loop over each fold
    for train_index, test_index in kf.split(X):
        # Split the data into training and test sets
        X_train, X_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        theta = Ridge_Regression_closed_poly(X_train, y_train, lamb)
        poly_param.append(theta)
        #predictions
        y_train_pred = predict_ridge_poly (X_train, theta)
        y_test_pred = predict_ridge_poly (X_test, theta)
        # Compute the Mean Squared Error (MSE) for both training and test sets -> Need to actually write formula
        train_mse = mean_squared_error(y_train_pred, y_train)
        test_mse = mean_squared_error(y_test_pred, y_test)
        # Append the errors to the lists
        train_errors.append(train_mse)
        test_errors.append(test_mse)
    # Calculate the average error across all folds
    avg_train_error = np.mean(train_errors)
    avg_test_error = np.mean(test_errors)
    print(f"Average training MSE: {avg_train_error}")
    print(f"Average test MSE: {avg_test_error}")

#Apply functions
op_lamb_poly = np.logspace(1, 7, num=13)  # Lambda values from 10^1 to 10^7
best_lamb_poly = ridge_cross_validation_best_poly(x_poly, Y, op_lamb_poly)
print(f"Best lambda value was: {best_lamb_poly}")
# Evaluate the performance with the best lambda
ridge_cross_validation_poly(x_poly, Y, best_lamb_poly)

Best lambda value was: 10000000.0
Average training MSE: 18.882249988235237
Average test MSE: 34.531693305166065


In [498]:
#Activity 7. Repeat Multivariate Linear Regression using the Gradient Descent method
X_normalized = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
X_b = np.c_[np.ones((X_normalized.shape[0], 1)), X_normalized]


def predict(X, theta):
    return np.dot(X, theta)

# Function to compute the cost (MSE)
def compute_cost(X_c, y, theta): #x_b test
    m = len(y)
    y_pred = predict (X_c, theta)
    cost = (1/(2*m)) * np.sum((y_pred - y)**2)
    return cost

# Gradient descent function
def gradient_descent(X_c, y, theta, learning_rate, iterations):
    m = len(y)
    cost_history = []

    for i in range(iterations):
        y_pred = predict (X_c, theta)
        error = y_pred - y
        gradients = (1/m) * X_c.T.dot(error)
        theta = theta - learning_rate * gradients
        cost = compute_cost(X_c, y, theta)
        cost_history.append(cost)
    return theta, cost_history


#Call the function
list_thetas = []
list_cost = []
kf = KFold(n_splits=10)
# Initialize hyperparameters
iterations = 1000
learning_rate = 0.000001


# Train the model using KFold cross-validation
for train_index, test_index in kf.split(X_b):
    X_train, X_test = X_b[train_index], X_b[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    # Initialize theta (parameter vector)
    theta = np.random.randn(X_train.shape[1])
    # Perform gradient descent
    theta_optimal, cost_history = gradient_descent(X_train, y_train, theta, learning_rate, iterations)
    list_thetas.append(theta_optimal)
    list_cost.append(cost_history)
    ave_cost = np.mean (cost_history)
    # Print the final cost and optimized theta
print(f"Optimized Theta: {theta_optimal}")
print(f"Final Cost: {ave_cost}")


Optimized Theta: [-0.75756286  1.30948006 -0.866291   -0.76046423  1.12882979  1.11403624
  3.18510342  0.07480554  1.093951    0.21266962 -0.02047211 -0.62668221
 -0.05048897 -0.46597186]
Final Cost: 307.01117468232894


In [499]:
# Activity 8 Implement Lasso Regression (Text book Chapter 4)a.Cost function of Lasso Regression:
#remember to pass x_b argument becaus ethe predict function does not adjust the matrix
# Function to compute the cost (MSE)
# Function to compute the cost (MSE) with L1 penalty (Lasso)
def compute_cost_Lasso(X_c, y, theta, l1_penalty):
    m = len(y)
    y_pred = X_c.dot(theta)  # Prediction using the current theta
    error = y_pred - y
    mse = (1/(2*m)) * np.dot(error.T, error)  # Mean Squared Error (MSE)
    l1_penalty_term = l1_penalty * np.sum(np.abs(theta[1:]))  # Exclude the intercept from regularization
    return mse + l1_penalty_term

# Gradient descent function for Lasso regression
def gradient_descent(X_c, y, theta, learning_rate, iterations, l1_penalty):
    m = len(y)
    cost_history = []

    for i in range(iterations):
        # Predict y using current theta
        y_pred = X_c.dot(theta)
        # Compute error
        error = y_pred - y
        # Compute gradient (without L1 regularization for intercept)
        gradient = (1/m) * X_c.T.dot(error)
        # Update theta (Apply L1 regularization using soft thresholding)
        theta = theta - learning_rate * gradient
        theta[1:] = np.sign(theta[1:]) * np.maximum(np.abs(theta[1:]) - learning_rate * l1_penalty, 0) 
        cost = compute_cost_Lasso(X_c, y, theta, l1_penalty)
        cost_history.append(cost)
        avg_cost = np.mean(cost_history)
        # Compute average theta (excluding the bias term if necessary)
        avg_theta = np.mean(theta)
        
    #print(avg_cost, avg_theta)

    return theta, cost_history

l1_penalty = 0.1

#gradient_descent(X_b, Y, theta, learning_rate, iterations, l1_penalty)

alpha = 0.1  
r = 0.5      
learning_rate = 0.01
num_iters = 1000

kf = KFold(n_splits=10, shuffle=True, random_state = 42)
ave_train = []
ave_test = []

for train_index, test_index in kf.split(X_b):
    X_train, X_test = X_b[train_index], X_b[test_index]
    y_train, y_test = Y[train_index], Y[test_index]

    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_test = np.c_[np.ones((X_test.shape[0], 1)), X_test]
    
    # Initialize theta with random values
    theta = np.random.randn(X_train.shape[1])
    
    # Use X_train and y_train for training
    theta, _ = gradient_descent(X_train, y_train, theta, learning_rate, iterations, l1_penalty)
    
    y_train_pred = predict(X_train, theta)
    y_test_pred = predict(X_test, theta)
    
    train_mse = np.mean((y_train - y_train_pred)**2)
    test_mse = np.mean((y_test - y_test_pred)**2)
    
    ave_train.append(train_mse)
    ave_test.append(test_mse)

print(f"Average Training MSE (Elastic Net): {np.mean(ave_train)}")
print(f"Average Test MSE (Elastic Net): {np.mean(ave_test)}")



Average Training MSE (Elastic Net): 22.675293975256317
Average Test MSE (Elastic Net): 24.17295639525117


In [500]:
#Activity 9 Implement Elastic Net (Text book Chapter 4) a.Cost Function of Elastic Net:

def cost_elastic_net(X_c, y, theta, alpha, r):
    m = len(y)
    predictions = predict(X_c, theta)
    error = (1/(2*m)) * np.sum((predictions - y)**2)
    penalty1 = r * alpha * np.sum(np.abs(theta))
    penalty2 = (1 - r) / 2 * alpha * np.sum(theta**2)
    return error + penalty1 + penalty2

def elastic_gradient_descent(X_c, y, theta, alpha, r, learning_rate, num_iters):
    m = len(y)
    cost_history = []
    for _ in range(num_iters):
        predictions = predict(X_c, theta)
        # Gradient of MSE (standard gradient descent)
        gradient_mse = (1/m) * X_c.T.dot(predictions - y)
        # L1 penalty gradient (Lasso part)
        gradient_l1 = r * alpha * np.sign(theta)
        # L2 penalty gradient (Ridge part)
        gradient_l2 = (1 - r) * alpha * theta
        # Full gradient with both L1 and L2 regularization terms
        gradient = gradient_mse + gradient_l1 + gradient_l2
        # gradient descent step
        theta = theta - learning_rate * gradient
        cost = cost_elastic_net(X_c, y, theta, alpha, r)
        cost_history.append(cost)
    return theta, cost_history

#remember to pass x_b argument becaus ethe predict function does not adjust the matrix
alpha = 2
r = 0.1      
learning_rate = 0.000001
num_iters = 100000

kf = KFold(n_splits=10)
ave_train = []
ave_test = []

for train_index, test_index in kf.split(X_b):
    X_train, X_test = X_b[train_index], X_b[test_index]
    y_train, y_test = Y[train_index], Y[test_index]
    
    theta = np.random.randn(X_train.shape[1])
    
    theta1, _ = elastic_gradient_descent(X_train, y_train, theta, alpha, r, learning_rate, num_iters)
    
    y_train_pred = predict(X_train, theta1)
    y_test_pred = predict(X_test, theta1)
    
    train_mse = np.mean((y_train - y_train_pred)**2)
    test_mse = np.mean((y_test - y_test_pred)**2)
    
    ave_train.append(train_mse)
    ave_test.append(test_mse)

print(f"Average Training MSE (Elastic Net): {np.mean(ave_train)}")
print(f"Average Test MSE (Elastic Net): {np.mean(ave_test)}")



Average Training MSE (Elastic Net): 493.7420009955441
Average Test MSE (Elastic Net): 509.277903374346


In [501]:
# Activity 10 If you are given a choice of predicting future housing prices using one of the modelsyou have learned above 
#(those optimized with gradient descent), which one wouldyou choose and why? State the parameters of that model.

#After analyzing the Mean Test Errors I came to the realization that the Lasso Regressation had the least error. Although this could be attributed
#to different reasons, for the sake of simplicity, this was the main factor that I am considering to select the Lasso model to predict the housing 
#prices. The parameters are alpha = 0.1  
#r = 0.5      
#learning_rate = 0.01
#num_iters = 1000
#l1_penalty = 0.1

